In [19]:
import os
import re
import pandas as pd

In [10]:
SCAN_BASE_DIR = 'First Law Scans Cropped/'
SCAN_DIRS = [SCAN_BASE_DIR + subdir + '/' for subdir in os.listdir(SCAN_BASE_DIR)]

In [66]:
counter = 0

# Regex for letter number heading
letter_number_pattern = re.compile('^\d+\..*$')

# Initialize empty dataframe for storing segmented text
df = pd.DataFrame()

for volume_dir in SCAN_DIRS:
    id2filename = {}
    
    # Associate each filename with its numbered id
    # so we can iterate in order through the files
    for filename in os.listdir(volume_dir):
        if '.txt' in filename:
            id = filename.split('_')[0]
            id2filename[int(id)] = filename
    
    for filename in id2filename.values():
        with open(volume_dir + filename, 'r') as f:
            contents = ""
            
            # TODO: Figure out why this hacky workaround is needed for reading the txt file
            # Likely Jupyter environment issue
            for line in f.readlines():
                contents += line
            
            contents_list = contents.split('\r')
            
            ptr = 0
            num_lines = len(contents_list)

            while ptr < num_lines:
                # Look for section heading
                if letter_number_pattern.match(contents_list[ptr]):
                    section_title = contents_list[ptr]
                    # Initialize string buffer for storing text for this section
                    text_buffer = ''
                    ptr += 1

                    # Move pointer forward until next section heading or end of file encountered
                    while ptr < num_lines and letter_number_pattern.match(contents_list[ptr]) is None:
                        text_buffer += contents_list[ptr]
                        ptr += 1

                    # Append to dataframe
                    df = df.append({
                                    'filename': filename,
                                    'section_title': section_title,
                                    'text': text_buffer
                                    }, 
                                    ignore_index=True)

                # Otherwise advance pointer
                else:
                    ptr += 1


In [73]:
# Export segmented text to csv
df.to_csv('csv/segmented.csv', index=False)